In [164]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf

In [165]:
# manually defining the stopwords
stop_words = ["a","about","above","after","again","against","all","am","an","and","any","are","as","at","be","because","been","before","being","below","between","both","but","by","could","did","do","does","doing","down","during","each","few","for","from","further","had","has","have","having","he","he’d","he’ll","he’s","her","here","here’s","hers","herself","him","himself","his","how","how’s","I","I’d","I’ll","I’m","I’ve","if","in","into","is","it","it’s","its","itself","let’s","me","more","most","my","myself","nor","of","on","once","only","or","other","ought","our","ours","ourselves","out","over","own","same","she","she’d","she’ll","she’s","should","so","some","such","than","that","that’s","the","their","theirs","them","themselves","then","there","there’s","these","they","they’d","they’ll","they’re","they’ve","this","those","through","to","too","under","until","up","very","was","we","we’d","we’ll","we’re","we’ve","were","what","what’s","when","when’s","where","where’s","which","while","who","who’s","whom","why","why’s","with","would","you","you’d","you’ll","you’re","you’ve","your","yours","yourself","yourselves"]
number_of_classes = 2 # love and sad

In [143]:
# data preprocessing part
def data_preprocessing(train):
    x_train = train.loc[:,['Document']]
    y_train = train.loc[:,['Sentiment']]
    y_train = y_train['Sentiment'].astype('category')
    y_train.cat.categories = range(y_train.nunique())
    y_train = np.ravel(y_train)
    x_train = list(x_train.Document)
    
    return (x_train,y_train)

In [173]:
def create_corpus(data):
    words_corpus = set()
    for row in data:
        row = re.sub(r'^[a-z][A-z]', '', row)
        row = row.split()
        row = [word for word in row if not word in stop_words]
        for word in row:
            words_corpus.add(word)
    
    words_corpus = list(words_corpus)
    return words_corpus

def count_vectorize(data,words_corpus):
    
    word_count = [[0]*len(words_corpus) for i in range(len(data))]
    i=0
    for row in data:
        row = re.sub(r'^[a-z][A-z]', '', row)
        row = row.split()
        row = [word for word in row if not word in stop_words]
        for word in row:
            if not word in stop_words:
                word_count[i][words_corpus.index(word)] += 1
        i+=1
  
    return word_count

In [145]:
def one_hot(data):
    one_hot_data = []
    for x in data:
        temp = np.zeros(number_of_classes)
        temp[x]=1
        one_hot_data.append(temp)
    return one_hot_data

In [146]:
def neural_network(x_train,y_train):
    
    epoch = 50
    batch_size = 20
    learning_rate = 0.01
    seed = 100
    
    input_neurons = len(x_train[0])
    hidden_neurons = 100
    output_neurons = number_of_classes
    x = tf.placeholder(tf.float32,[None,input_neurons])
    y = tf.placeholder(tf.float32,[None,2])
    
    weights = {'hidden':tf.Variable(tf.random_normal([input_neurons,hidden_neurons],seed=seed)),
               'output':tf.Variable(tf.random_normal([hidden_neurons,output_neurons],seed=seed))}
    biases  = {'hidden':tf.Variable(tf.random_normal([hidden_neurons],seed=seed)),
               'output':tf.Variable(tf.random_normal([output_neurons],seed=seed))}
    
    hidden_layer = tf.add(tf.matmul(x,weights['hidden']),biases['hidden'])
    hidden_layer = tf.nn.relu(hidden_layer)
    
    output_layer = tf.add(tf.matmul(hidden_layer,weights['output']),biases['output'])
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=output_layer))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
    
    
    init = tf.initialize_all_variables()
    
    

In [197]:
def main(test):
    train = pd.read_csv('my_dataset.csv')
    # print(train.describe())
    x_train,y_train = data_preprocessing(train)
    
    # creating the word corpus using both test and train data
    x_train.extend(test)
    words_corpus = create_corpus(x_train)
    x_train.pop(-1)
    
    # creating count vectors
    x_train = count_vectorize(x_train,words_corpus)
    x_test = count_vectorize(test,words_corpus)
    
    # one hot encoding of the test data
    # for example: [0] -> [1,0] and [1] -> [0,1]
    y_train = one_hot(y_train)


if __name__ == '__main__':
    
    test_sentence = []
    test_sentence.append(raw_input('Enter the test sentence : '))
    # calling the main function
    main(test_sentence)

Enter the test sentence : my name is ashutosh


In [192]:
x=[1,2]
y=x.pop(-1)
y

2